In an effort to start learning some patterns in ML, I decided to go with small steps and reimplement some patterns in OCaml. Assumming the concept of `list` doesn't exist yet we will reimplement a similar concept known as `sequence`:

In [3]:
type 'a seq =
  | End
  | Next of 'a * 'a seq
;;

type 'a seq = End | Next of 'a * 'a seq


We can define a sequence of zero elements like this:

In [4]:
let no_elements = End

val no_elements : 'a seq = End


Or the count from 1 to 3 like this:

In [5]:
let sample = Next(1, Next(2, Next(3, End)))

val sample : int seq = Next (1, Next (2, Next (3, End)))


### Implement basic operations

Length, Head, Tail, Cons

In [6]:
(* length operator *)
let length_seq x =
  let rec length' acc = function
    | End -> acc
    | Next (_, t) -> length' (acc + 1) t
  in
  length' 0 x
;;

length_seq sample;;

(* head operator *)
let head_seq = function
  | End -> raise (Invalid_argument "empty sequence")
  | Next (h, _) -> h
;;

head_seq sample;;

(* tail operator *)
let tail_seq = function
  | End -> raise (Invalid_argument "empty sequence")
  | Next (_, t) -> t
;;

tail_seq sample;;

(* cons operator *)
let cons_seq x l = Next(x, l) ;;
cons_seq 0 sample ;;

val length_seq : 'a seq -> int = <fun>


- : int = 3


val head_seq : 'a seq -> 'a = <fun>


- : int = 1


val tail_seq : 'a seq -> 'a seq = <fun>


- : int seq = Next (2, Next (3, End))


val cons_seq : 'a -> 'a seq -> 'a seq = <fun>


- : int seq = Next (0, Next (1, Next (2, Next (3, End))))


Now we can start implementing even more complex operations, like `nth`, `rev`, `append`

In [49]:
(* nth operator *)
let rec nth_seq n l = 
  if n < 0 then raise (Invalid_argument "nth") else
  match l with
    | End -> raise (Failure "nth")
    | Next (x, tl) -> if n = 0 then x else nth_seq (n - 1) tl
;;

nth_seq 0 sample ;;
nth_seq 2 sample ;;

(* rev operator *)
let rev_seq l =
  let rec rev' acc = function
    | End -> acc
    | Next (x, tl) -> rev' (cons_seq x acc) tl
  in
  rev' End l
;;

rev_seq sample ;;

(* append operator, this function is not tail recursive *)
(* the original in OCaml neither *)
let rec append_seq a b =
  match a with
   | End -> b
   | Next (x, tl) -> Next(x, ((append_seq [@tailcall]) tl b))
;;

append_seq sample (Next(4, Next(5, Next(6, End))))

(* We can make a recursive append *)
let append_seq' a b =
  let rec append' acc = function
    | End -> b
    | Next (x, t) -> append' (Next(x, acc)) t
  in
  append' End (rev_seq a)
;;

append_seq sample (Next(4, Next(5, Next(6, End)))) ;;

let (+:) a b = cons_seq a b ;;
0 +: sample

val nth_seq : int -> 'a seq -> 'a = <fun>


- : int = 1


- : int = 3


val rev_seq : 'a seq -> 'a seq = <fun>


- : int seq = Next (3, Next (2, Next (1, End)))


...

- : int seq = Next (1, Next (2, Next (3, Next (4, Next (5, Next (6, End))))))


val append_seq' : 'a seq -> 'b -> 'b = <fun>


- : int seq = Next (1, Next (2, Next (3, Next (4, Next (5, Next (6, End))))))


val ( +: ) : 'a -> 'a seq -> 'a seq = <fun>


- : int seq = Next (0, Next (1, Next (2, Next (3, End))))


Now we can start playing with functions and lists. What about `init`, `map` and, more importantly, `fold_right` and `fold_left`?

In [48]:
(* init operator *)
let init_seq n f =
  let rec init' n' acc =
    if n' = 0 then acc else init' (n' - 1) (cons_seq (f n') acc)
  in
  init' n End
;;

init_seq 3 (fun x -> x) ;;

(* map operator, not tail recursive *)
let rec map_seq f = function
  | Next (x, t) -> Next ((f x), ((map_seq [@tailcall]) f t))
  | _ -> End
;;

map_seq (fun x -> x * 2) sample ;;

(* map operator, now tail recursive *)
let map_seq f l =
  let rec map_seq' acc = function
    | End -> acc
    | Next (x, t) -> map_seq' (Next((f x), acc)) t
  in
  rev_seq (map_seq' End l)
;;

map_seq (fun x -> x * 2) sample

val init_seq : int -> (int -> 'a) -> 'a seq = <fun>


- : int seq = Next (1, Next (2, Next (3, End)))


File "[48]", line 13, characters 32-59:
Warning 51: expected tailcall


val map_seq : ('a -> 'b) -> 'a seq -> 'b seq = <fun>


- : int seq = Next (2, Next (4, Next (6, End)))


val map_seq : ('a -> 'b) -> 'a seq -> 'b seq = <fun>


- : int seq = Next (2, Next (4, Next (6, End)))


Now it is time for the `fold` functions. The fold functions is just a nicer way to say "aggregate" functions.

In [20]:
let fold_right_seq f a l =
  let rec fold' cur = function
    | End -> cur
    | Next (x, t) -> fold' (f x cur) t
  in
  fold' a l
;;

fold_right_seq (fun a b -> a + b) 0 sample

val fold_right_seq : ('a -> 'b -> 'b) -> 'b -> 'a seq -> 'b = <fun>


- : int = 6


Fold is super powerful, we can even express map as a fold function!

In [25]:
let fold_map_seq f l =
  let map' x acc = cons_seq (f x) acc
  in
  rev_seq (fold_right_seq map' End l)
;;
  
fold_map_seq (fun x -> x * 2) sample

val fold_map_seq : ('a -> 'b) -> 'a seq -> 'b seq = <fun>


- : int seq = Next (2, Next (4, Next (6, End)))
